**Задание 1**

Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце 'region' пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим 'undefined'.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {

    'Центр': ['москва', 'тула', 'ярославль'],

    'Северо-Запад': ['петербург', 'псков', 'мурманск'],

    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']

}

Результат классификации запишите в отдельный столбец region.

In [28]:
import pandas as pd

In [29]:
keywords = pd.read_csv('keywords.csv')
keywords.head()

,keyword,shows
0,вк,64292779
1,одноклассники,63810309
2,порно,41747114
3,ютуб,39995567
4,вконтакте,21014195


In [30]:
geo_data = {

    'Центр': ['москва', 'тула', 'ярославль'],

    'Северо-Запад': ['петербург', 'псков', 'мурманск'],

    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']

}

In [31]:
def region(row):
    """гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону"""
    n=0
    s=0
    for key in geo_data.values():
        for i in range(len(key)):
            if key[i] in row['keyword']:    
                s=s+1      
                return list(geo_data)[n]                   
        n = n +1
    if s==0:
        return 'undefined'

In [32]:
keywords['region'] = keywords.apply(region, axis=1)
keywords.head()

,keyword,shows,region
0,вк,64292779,undefined
1,одноклассники,63810309,undefined
2,порно,41747114,undefined
3,ютуб,39995567,undefined
4,вконтакте,21014195,undefined


In [33]:
keywords['region'].value_counts()

undefined         99253
Центр               387
Северо-Запад        266
Дальний Восток       94
Name: region, dtype: int64

**Задание 2**

Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
    - оценка 2 и меньше - низкий рейтинг
    - оценка 4 и меньше - средний рейтинг
    - оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [34]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [35]:
average_ratings = ratings.groupby('movieId').mean().reset_index()
average_ratings.head()

,movieId,userId,rating,timestamp
0,1,338.558704,3.872470,1.103116e+09
1,2,318.906542,3.401869,1.069321e+09
2,3,374.423729,3.161017,9.662429e+08
3,4,355.538462,2.384615,9.277797e+08
4,5,320.785714,3.267857,9.967201e+08


In [36]:
def film_class(row):
    #классифицирует фильмы в зависимости от среднего рейтинга
    
    if row['rating'] <= 2:
        return 'низкий рейтинг'
    elif 2<row['rating']<=4:
        return 'средний рейтинг'    
    else:
        return 'высокий рейтинг'

In [37]:
average_ratings['class'] = average_ratings.apply(film_class, axis=1)
average_ratings.head()

,movieId,userId,rating,timestamp,class
0,1,338.558704,3.872470,1.103116e+09,средний рейтинг
1,2,318.906542,3.401869,1.069321e+09,средний рейтинг
2,3,374.423729,3.161017,9.662429e+08,средний рейтинг
3,4,355.538462,2.384615,9.277797e+08,средний рейтинг
4,5,320.785714,3.267857,9.967201e+08,средний рейтинг


In [11]:
average_ratings['class'].value_counts()

средний рейтинг    6821
высокий рейтинг    1272
низкий рейтинг      973
Name: class, dtype: int64

**Задание 3**

Посчитайте среднее значение Lifetime киноманов (пользователи, которые поставили 100 и более рейтингов). Под Lifetime понимается разница между максимальным и минимальным значением timestamp для каждого пользователя. Ответ дайте в днях.

In [38]:
ratings_100 = ratings.groupby('userId').count().sort_values('userId').reset_index()
ratings_diff = ratings.groupby('userId').agg([min, max]).sort_values('userId').reset_index()

In [39]:
len(ratings_100) == len(ratings_diff)

True

In [40]:
joined = ratings_100.merge(ratings_diff, on='userId', how='left')
joined.head()

,userId,movieId,rating,timestamp,"(movieId, min)","(movieId, max)","(rating, min)","(rating, max)","(timestamp, min)","(timestamp, max)"
0,1,20,20,20,31,3671,1.0,4.0,1260759108,1260759205
1,2,76,76,76,10,720,1.0,5.0,835355395,835356246
2,3,51,51,51,60,84236,2.0,5.0,1298861589,1298932787
3,4,204,204,204,10,4006,1.0,5.0,949778714,949982274
4,5,100,100,100,3,48385,1.5,5.0,1163373044,1163375145


In [41]:
len(ratings_100) == len(joined)

True

In [42]:
joined.columns = ['userId', 'movieId', 'rating', 'timestamp', 'movieId_min', 'movieId_max', 'rating_min', 'rating_max', 'timestamp_min', 'timestamp_max']
joined.head()

,userId,movieId,rating,timestamp,movieId_min,movieId_max,rating_min,rating_max,timestamp_min,timestamp_max
0,1,20,20,20,31,3671,1.0,4.0,1260759108,1260759205
1,2,76,76,76,10,720,1.0,5.0,835355395,835356246
2,3,51,51,51,60,84236,2.0,5.0,1298861589,1298932787
3,4,204,204,204,10,4006,1.0,5.0,949778714,949982274
4,5,100,100,100,3,48385,1.5,5.0,1163373044,1163375145


In [43]:
joined['life'] = joined['timestamp_max'] - joined['timestamp_min']
joined.head()

,userId,movieId,rating,timestamp,movieId_min,movieId_max,rating_min,rating_max,timestamp_min,timestamp_max,life
0,1,20,20,20,31,3671,1.0,4.0,1260759108,1260759205,97
1,2,76,76,76,10,720,1.0,5.0,835355395,835356246,851
2,3,51,51,51,60,84236,2.0,5.0,1298861589,1298932787,71198
3,4,204,204,204,10,4006,1.0,5.0,949778714,949982274,203560
4,5,100,100,100,3,48385,1.5,5.0,1163373044,1163375145,2101


In [44]:
film_fans = joined[ (joined['movieId']>=100) ]

In [18]:
film_fans['life'].mean() / 24 / 3600

455.2285713719898

**Задание 4**

Есть мнение, что "раньше снимали настоящее кино, не то что сейчас". Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из материалов занятия. Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

1. В переменную years запишите список из всех годов с 1950 по 2010.

2. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:
    - для каждой строки пройдите по всем годам списка years
    - если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
    - если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год

3. Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец 'year'

4. Посчитайте средний рейтинг всех фильмов для каждого значения столбца 'year' и отсортируйте результат по убыванию рейтинга

In [45]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [46]:
movies = pd.read_csv('ml-latest-small/movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [47]:
years = list(range(1950,2011))

In [48]:
def production_year(row):
    #каждой строке из названия фильма выставляет год выпуска
    for year in years:
        if str(year) in row['title']:
            return year
    return '1900'

In [49]:
movies['year'] = movies.apply(production_year, axis=1)
movies.head()

,movieId,title,genres,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II (1995),Comedy,1995


In [50]:
joined = ratings.merge(movies, on='movieId', how='left')
joined.head()

,userId,movieId,rating,timestamp,title,genres,year
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama,1995
1,1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical,1900
2,1,1061,3.0,1260759182,Sleepers (1996),Thriller,1996
3,1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller,1981
4,1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama,1989


In [51]:
len(ratings) == len(joined)

True

In [52]:
joined.groupby('year').mean().sort_values('rating', ascending=False).head()

,userId,movieId,rating,timestamp
year,,,,
1957,360.933544,2750.965190,4.014241,1.083707e+09
1972,359.694878,3983.538976,4.011136,1.122759e+09
1952,346.394737,4107.796053,4.000000,1.090512e+09
1954,358.228324,2867.661850,3.994220,1.070591e+09
1951,347.106996,2605.588477,3.983539,1.052714e+09
